# AI 201 Programming Assignment 1
## A* Algorithm Implementation

Submitted by: 
Jan Lendl R. Uy, 2019-00312

In [1]:
# CONSTANTS

# Relative path to the input file
INPUT_FILE_PATH = "astar_in.txt"

# Set a maximum number of iterations for A* to avoid infinite 
# loop if there is no solution
MAX_A_STAR_ITERATIONS = 50000

## File Handling
This function reads an input .txt file containing the start and goal states of the puzzle then converts the puzzle contents into a two-dimensional tuple of tile values. The input file is stored in the same directory as the notebook file with the filename of "astar_in.txt".

In [2]:
def read_file(path):
    
    with open(path, "r") as file:
        lines = file.readlines()

    # Initialize lists to store the start state and goal state
    start_state = []
    goal_state = []

    # Initialize a flag to keep track if following tiles belong to the start
    # or the goal state
    reading_part = None

    for line in lines:
        line = line.strip() # Remove leading/trailing whitespace
        if line == "start":
            reading_part = "start"
        elif line == "goal":
            reading_part = "goal"
        elif line and reading_part:
            # Convert line into a list, handling "*" and converting numbers to integers
            line_list = [int(x) if x.isdigit() else x for x in line.split()]
            if reading_part == "start":
                start_state.append(line_list)
            elif reading_part == "goal":
                goal_state.append(line_list)
                
    start_state = tuple(tuple(row) for row in start_state)
    goal_state = tuple(tuple(row) for row in goal_state)

    return start_state, goal_state

In [3]:
start_state, goal_state = read_file(INPUT_FILE_PATH)
print(f"Start State: {start_state}")
print(f"Goal State: {goal_state}")

Start State: ((2, 1, 6), (4, '*', 8), (7, 5, 3))
Goal State: ((1, 2, 3), (8, '*', 4), (7, 6, 5))


## PuzzleBoard
This class encapsulates the puzzle as a tuple of tuples (i.e. 2D tuple).

Among the operators overridden in this class are the following:
- `__str__` to "prettify" the printing of the 8-puzzle
- `__eq__` and `__hash__` for checking of equal board contents

Aside from this, the following helper methods are implemented to simplify the operations of the game:
- `convert_board_to_string()` which reads the two-dimensional tuple representation of the puzzle and converts this into a prettier format
- `_find_empty_tile()` which looks for the empty tile denoted by a "*" in the puzzle
- `get_possible_moves()` which determines all of the possible moves of the empty tile based on its current position in the puzzle

The heuristic functions are also found in this class as methods of the puzzle. These are:
- `get_misplaced_tiles()` which, as its name implies, counts the number of misplaced in the current puzzle in reference to the goal puzzle state
- `get_manhattan_distance()` which obtains the Manhattan Distance of each tile from its current position to its desired position
- `get_nilsson_sequence_score()` which obtains the Nilsson Sequence Score by computing the sequence score multiplied by 3 and summed to the Manhattan Distance of each tile from the current position to the desired position

It is worth noting that computing the Nilsson Sequence Score has a caveat. It is only designed to work for an 8-Puzzle. Other puzzle sizes will not work for this heuristic function programmed below.

In [4]:
class PuzzleBoard:
    
    def __init__(self, board):
        self.board = board
        
        # Assume that the puzzle is a square
        self.rows = len(board)
        self.cols = len(board[0])
        
        self.board_as_string = self.convert_board_to_string()
        
    def convert_board_to_string(self):
        if not self.board:
            return "Empty board"
        
        board = ""

        for i in range(self.rows):
            board += "| "
            for j in range(self.cols):
                tile = self.board[i][j]
                board += str(tile)
                board += " | "
            board += "\n"
            
        return board

    def __str__(self):
                    
        return self.board_as_string
    
    def __eq__(self, other):
        if not isinstance(other, PuzzleBoard):
            return False
        return self.board == other.board

    def __hash__(self):
        return hash(self.board)
    
    def _find_empty_tile(self):
        for i, row in enumerate(self.board):
            for j, tile in enumerate(row):
                if tile == "*":
                    return (i, j)
        raise ValueError("No empty tile found in the puzzle")
    
    def get_possible_moves(self):
        empty_row, empty_col = self._find_empty_tile()
        possible_moves = []
        
        # Possible moves: up, down, left, right
        # Tuples indicate the numbers added to the coordinates of the empty tile
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        
        for direction in directions:
            new_row = empty_row + direction[0]
            new_col = empty_col + direction[1]
            
            # Check if the new position is within the board
            if 0 <= new_row < len(self.board) and 0 <= new_col < len(self.board[0]):
                # Create a new board configuration with the move applied
                new_board = [list(row) for row in self.board]  # Convert tuple of tuples to list of lists
                new_board[empty_row][empty_col], new_board[new_row][new_col] = new_board[new_row][new_col], new_board[empty_row][empty_col]
                new_board = tuple(tuple(row) for row in new_board)  # Convert back to tuple of tuples
                
                possible_moves.append(new_board)
        
        return possible_moves
    
    # Helper methods useful for computing the different heuristic functions h(n)
    
    def get_misplaced_tiles(self, goal_puzzle):
        misplaced = 0
        goal_state = goal_puzzle.board
        for i in range(self.rows):
            for j in range(self.cols):
                if self.board[i][j] != "*" and self.board[i][j] != goal_state[i][j]:
                    misplaced += 1
        return misplaced
    
    def get_manhattan_distance(self, goal_puzzle):
        total_distance = 0
        goal_state = goal_puzzle.board

        for i in range(self.rows):
            for j in range(self.cols):
                if self.board[i][j] != "*":
                    # Find the tile's position in the goal state
                    for gi in range(self.rows):
                        for gj in range(self.cols):
                            if self.board[i][j] == goal_state[gi][gj]:
                                # Calculate Manhattan distance for this tile
                                distance = abs(i - gi) + abs(j - gj)
                                total_distance += distance
                                break
                        else:
                            continue
                        break

        return total_distance
    
    def get_nilsson_sequence_score(self, goal_puzzle):
        current_board = self.board
        goal_board = goal_puzzle.board

        manhattan_dist = self.get_manhattan_distance(goal_puzzle)
        
        # Flatten the boards for easier index computation
        current_board_flat = [tile for row in current_board for tile in row]
        goal_board_flat = [tile for row in goal_board for tile in row]
        
        # The correct clockwise ordering of the tiles
        goal_order = goal_board_flat[0:3] + [goal_board_flat[5], goal_board_flat[8], goal_board_flat[7], goal_board_flat[6], goal_board_flat[3]]
        current_order = current_board_flat[0:3] + [current_board_flat[5], current_board_flat[8], current_board_flat[7], current_board_flat[6], current_board_flat[3]]

        # Calculate sequence score
        goal_pairs = []
        current_pairs = []
        sequence_score = 0
        for i in range(-1, -len(goal_order), -1):
            current_pair = (current_order[i], current_order[i+1])
            goal_pair = (goal_order[i], goal_order[i+1])
            goal_pairs.append(goal_pair)
            if current_pair[0] == "*":
                continue
            current_pairs.append(current_pair)

        for pair in current_pairs:
            if pair not in goal_pairs:
                sequence_score += 2

        # Check the center square, add 1 if it is not the empty square
        if current_board_flat[4] != "*":
            sequence_score += 1
        
        # Multiply the sequence score by 3 and add the Manhattan distance
        return 3 * sequence_score + manhattan_dist, manhattan_dist, sequence_score

In [5]:
start_puzzle = PuzzleBoard(start_state)
goal_puzzle = PuzzleBoard(goal_state)
print(f"Start Puzzle: \n{start_puzzle}")
print(f"Goal Puzzle: \n{goal_puzzle}")

Start Puzzle: 
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Goal Puzzle: 
| 1 | 2 | 3 | 
| 8 | * | 4 | 
| 7 | 6 | 5 | 



## PuzzleSearchNode
This class is used for instantiating a node that will be used in creating the search tree for A* search. It stores the following attributes:
- `self.puzzle` stores the state
- `self.parent` stores the predecessor node of the current node
- `self.g`, `self.h`, and `self.f` store the path cost, estimated cost, and the total estimated cost, respectively

Among the operators overridden in this class are the following:
- `__lt__` that compares the "value" of two nodes based on the total estimated path cost f(n) 
- `__eq__` and `__hash__` for checking of equal node (i.e. similar board configuration)

There are two methods that enable the heuristic search operations:
- `expand()` calls the get_possible_moves() method of PuzzleBoard and creates node out of each possible state (i.e. puzzle configurations)
- `compute_costs` is an update method for computing the estimated cost h(n) and the total estimated path cost f(n)

In [6]:
class PuzzleSearchNode:

    def __init__(self, puzzle, heuristic, g=0, parent=None):
        self.puzzle = puzzle
        self.parent = parent
        self.heuristic = heuristic
        self.g = g  # Path cost from start node to current node
        self.h = 0  # Heuristic estimate from current node to goal
        self.f = 0  # Total estimated path cost (g + h)
        
        self.p = 0 # Manhattan Distance, for the purpose of printing P(n) of Nilsson heuristic
        self.s = 0 # Sequence Score, for the purpose of printing S(n) of Nilsson heuristic

    def expand(self):
        expanded = []
        for new_puzzle in self.puzzle.get_possible_moves():
            child_path_cost = self.g + 1 # Increment path cost of child node by 1
            child_puzzle = PuzzleBoard(new_puzzle) # Initialize a new instance for each 8-puzzle state 
            child = PuzzleSearchNode(child_puzzle, self.heuristic, child_path_cost, self)
            expanded.append(child)
        return expanded

    def compute_costs(self, goal_node):
        goal_puzzle = goal_node.puzzle
        if self.heuristic == "misplaced":
            self.h = self.puzzle.get_misplaced_tiles(goal_puzzle)
        elif self.heuristic == "manhattan":
            self.h = self.puzzle.get_manhattan_distance(goal_puzzle)
        elif self.heuristic == "nilsson":
            self.h, self.p, self.s = self.puzzle.get_nilsson_sequence_score(goal_puzzle)
        self.f = self.g + self.h

    def goal_test(self, goal_node):
        return self == goal_node

    def __lt__(self, other):
        return self.f < other.f

    def __eq__(self, other):
        return isinstance(other, PuzzleSearchNode) and self.puzzle == other.puzzle

    def __hash__(self):
        return hash(self.puzzle)

## A* Search
Using the class definitions for the 8-Puzzle and the search node, the A* algorithm is implemented as shown below, following the same algorithm outlined in slide 21 of Lecture 3B.

In [7]:
def print_node_costs(node, search_cost, heuristic):
    print(f"f(n) = {node.f}, g(n) = {node.g}, h(n) = {node.h}")
    if heuristic == "nilsson":
        print(f"Nilsson heuristic-specific values: P(n) = {node.p}, S(n) = {node.s}")
    print(f"Cumulative search cost = {search_cost}")
    print(f"Current node:\n{node.puzzle}")

In [8]:
def astar_search(start_puzzle, goal_puzzle, heuristic="misplaced"):
    
    start_node = PuzzleSearchNode(start_puzzle, heuristic)
    goal_node = PuzzleSearchNode(goal_puzzle, heuristic)
    
    # Step 1: Compute initial costs for the start node
    start_node.compute_costs(goal_node)
    
    # Step 1: Initialize open list (and closed list)
    open_list = [start_node]
    closed_list = []
    
    search_cost = 0 # Keep track of search cost in terms of nodes generated

    # print(f"Initial estimated path cost f = {start_node.f}")
    
    for i in range(MAX_A_STAR_ITERATIONS):
        
        # Step 2: Terminate if open list is empty
        if len(open_list) == 0:
            return "Failed to find the goal node!"
        
        # Step 3: Find the node with the lowest f value
        # Remove this node from open list and then add to the closed list
        current_node = min(open_list)
        open_list.remove(current_node)
        closed_list.append(current_node)
                
        # Step 4: If the current node is the goal node, return the current node
        if current_node.goal_test(goal_node):
            print("Goal node found!")
            return i, search_cost, current_node
                
        # Step 5: Expand current node to get successor nodes
        # If no nodes from expansion, continue iteration
        expansion = current_node.expand()
        if len(expansion) < 1:
            print(f"Iteration {i+1}:")
            print_node_costs(current_node, search_cost, heuristic)
            continue
        
        search_cost += len(expansion) # Update search cost
                
        for successor_node in expansion:
            successor_node.compute_costs(goal_node)
            
            # Step 6 and 7: Handle successors
            existing_open_node = None
            existing_closed_node = None

            # Check if the successor node is in the open list
            if successor_node in open_list:
                existing_open_node = open_list[open_list.index(successor_node)]

            # Check if the successor node is in the closed list
            if successor_node in closed_list:
                existing_closed_node = closed_list[closed_list.index(successor_node)]
            
            if existing_open_node and existing_closed_node:
                # Node exists in both lists, use the one with lower f value
                if existing_open_node < existing_closed_node:
                    existing_node = existing_open_node
                else:
                    existing_node = existing_closed_node
            elif existing_open_node:
                existing_node = existing_open_node
            elif existing_closed_node:
                existing_node = existing_closed_node
            else:
                existing_node = None
            
            if existing_node:
                if successor_node < existing_node:
                    # Update the costs of existing node for the node with lower f
                    existing_node.g = successor_node.g
                    existing_node.f = successor_node.f
                    existing_node.parent = current_node
                    
                    # Move from closed list to open list if f values are lower
                    if existing_node in closed_list:
                        closed_list.remove(existing_node)
                        open_list.append(existing_node)
            else:
                # New node, add to open list
                open_list.append(successor_node)
        
        print(f"Iteration {i+1}:")
        print_node_costs(current_node, search_cost, heuristic)
        
    return f"Failed to find a solution within {MAX_A_STAR_ITERATIONS} iterations!"

def get_solution(goal_node):
    solution = [goal_node]
    current_node = goal_node
    while True:
        current_node = current_node.parent
        if current_node == None:
            break
        solution.append(current_node)
    solution.reverse()
    return solution

### A* Search using Number of Misplaced Tiles as the Heuristic

In [9]:
iters_misplaced, search_cost_misplaced, result_misplaced = astar_search(start_puzzle, goal_puzzle)
print(f"Solution found for {iters_misplaced} iterations of A* search. The heuristic function is the number of misplaced tiles.")
print(f"Search cost: {search_cost_misplaced} nodes generated")

Iteration 1:
f(n) = 7, g(n) = 0, h(n) = 7
Cumulative search cost = 4
Current node:
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Iteration 2:
f(n) = 8, g(n) = 1, h(n) = 7
Cumulative search cost = 7
Current node:
| 2 | * | 6 | 
| 4 | 1 | 8 | 
| 7 | 5 | 3 | 

Iteration 3:
f(n) = 8, g(n) = 1, h(n) = 7
Cumulative search cost = 10
Current node:
| 2 | 1 | 6 | 
| 4 | 5 | 8 | 
| 7 | * | 3 | 

Iteration 4:
f(n) = 8, g(n) = 1, h(n) = 7
Cumulative search cost = 13
Current node:
| 2 | 1 | 6 | 
| * | 4 | 8 | 
| 7 | 5 | 3 | 

Iteration 5:
f(n) = 8, g(n) = 1, h(n) = 7
Cumulative search cost = 16
Current node:
| 2 | 1 | 6 | 
| 4 | 8 | * | 
| 7 | 5 | 3 | 

Iteration 6:
f(n) = 8, g(n) = 2, h(n) = 6
Cumulative search cost = 18
Current node:
| * | 2 | 6 | 
| 4 | 1 | 8 | 
| 7 | 5 | 3 | 

Iteration 7:
f(n) = 9, g(n) = 2, h(n) = 7
Cumulative search cost = 20
Current node:
| 2 | 6 | * | 
| 4 | 1 | 8 | 
| 7 | 5 | 3 | 

Iteration 8:
f(n) = 9, g(n) = 2, h(n) = 7
Cumulative search cost = 22
Current node:
| 2 | 1 

In [10]:
solution_astar_misplaced = get_solution(result_misplaced)
i = 1
for node in solution_astar_misplaced:
    print(f"Step {i}: f(n) = {node.f}, g(n) = {node.g}, h(n) = {node.h} \n{node.puzzle}")
    i += 1

Step 1: f(n) = 7, g(n) = 0, h(n) = 7 
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Step 2: f(n) = 8, g(n) = 1, h(n) = 7 
| 2 | 1 | 6 | 
| 4 | 8 | * | 
| 7 | 5 | 3 | 

Step 3: f(n) = 9, g(n) = 2, h(n) = 7 
| 2 | 1 | * | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Step 4: f(n) = 10, g(n) = 3, h(n) = 7 
| 2 | * | 1 | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Step 5: f(n) = 11, g(n) = 4, h(n) = 7 
| 2 | 8 | 1 | 
| 4 | * | 6 | 
| 7 | 5 | 3 | 

Step 6: f(n) = 12, g(n) = 5, h(n) = 7 
| 2 | 8 | 1 | 
| 4 | 6 | * | 
| 7 | 5 | 3 | 

Step 7: f(n) = 13, g(n) = 6, h(n) = 7 
| 2 | 8 | 1 | 
| 4 | 6 | 3 | 
| 7 | 5 | * | 

Step 8: f(n) = 13, g(n) = 7, h(n) = 6 
| 2 | 8 | 1 | 
| 4 | 6 | 3 | 
| 7 | * | 5 | 

Step 9: f(n) = 13, g(n) = 8, h(n) = 5 
| 2 | 8 | 1 | 
| 4 | * | 3 | 
| 7 | 6 | 5 | 

Step 10: f(n) = 14, g(n) = 9, h(n) = 5 
| 2 | 8 | 1 | 
| * | 4 | 3 | 
| 7 | 6 | 5 | 

Step 11: f(n) = 15, g(n) = 10, h(n) = 5 
| * | 8 | 1 | 
| 2 | 4 | 3 | 
| 7 | 6 | 5 | 

Step 12: f(n) = 16, g(n) = 11, h(n) = 5 
| 8 | * | 1 | 
| 2 | 4 |

### A* Search using Manhattan Distance as the Heuristic

In [11]:
iters_manhattan, search_cost_manhattan, result_manhattan = astar_search(start_puzzle, goal_puzzle, "manhattan")
print(f"Solution found for {iters_manhattan} iterations of A* search. The heuristic function is Manhattan Distance.")
print(f"Search cost: {search_cost_manhattan} nodes generated")

Iteration 1:
f(n) = 12, g(n) = 0, h(n) = 12
Cumulative search cost = 4
Current node:
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Iteration 2:
f(n) = 12, g(n) = 1, h(n) = 11
Cumulative search cost = 7
Current node:
| 2 | 1 | 6 | 
| * | 4 | 8 | 
| 7 | 5 | 3 | 

Iteration 3:
f(n) = 12, g(n) = 1, h(n) = 11
Cumulative search cost = 10
Current node:
| 2 | 1 | 6 | 
| 4 | 8 | * | 
| 7 | 5 | 3 | 

Iteration 4:
f(n) = 12, g(n) = 2, h(n) = 10
Cumulative search cost = 12
Current node:
| 2 | 1 | * | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Iteration 5:
f(n) = 12, g(n) = 2, h(n) = 10
Cumulative search cost = 14
Current node:
| 2 | 1 | 6 | 
| 4 | 8 | 3 | 
| 7 | 5 | * | 

Iteration 6:
f(n) = 12, g(n) = 3, h(n) = 9
Cumulative search cost = 17
Current node:
| 2 | 1 | 6 | 
| 4 | 8 | 3 | 
| 7 | * | 5 | 

Iteration 7:
f(n) = 14, g(n) = 1, h(n) = 13
Cumulative search cost = 20
Current node:
| 2 | * | 6 | 
| 4 | 1 | 8 | 
| 7 | 5 | 3 | 

Iteration 8:
f(n) = 14, g(n) = 1, h(n) = 13
Cumulative search cost = 23
Current

In [12]:
solution_astar_manhattan = get_solution(result_manhattan)
i = 1
for node in solution_astar_manhattan:
    print(f"Step {i}: f(n) = {node.f}, g(n) = {node.g}, h(n) = {node.h} \n{node.puzzle}")
    i += 1

Step 1: f(n) = 12, g(n) = 0, h(n) = 12 
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Step 2: f(n) = 12, g(n) = 1, h(n) = 11 
| 2 | 1 | 6 | 
| 4 | 8 | * | 
| 7 | 5 | 3 | 

Step 3: f(n) = 12, g(n) = 2, h(n) = 10 
| 2 | 1 | * | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Step 4: f(n) = 14, g(n) = 3, h(n) = 11 
| 2 | * | 1 | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Step 5: f(n) = 16, g(n) = 4, h(n) = 12 
| 2 | 8 | 1 | 
| 4 | * | 6 | 
| 7 | 5 | 3 | 

Step 6: f(n) = 16, g(n) = 5, h(n) = 11 
| 2 | 8 | 1 | 
| 4 | 6 | * | 
| 7 | 5 | 3 | 

Step 7: f(n) = 16, g(n) = 6, h(n) = 10 
| 2 | 8 | 1 | 
| 4 | 6 | 3 | 
| 7 | 5 | * | 

Step 8: f(n) = 16, g(n) = 7, h(n) = 9 
| 2 | 8 | 1 | 
| 4 | 6 | 3 | 
| 7 | * | 5 | 

Step 9: f(n) = 16, g(n) = 8, h(n) = 8 
| 2 | 8 | 1 | 
| 4 | * | 3 | 
| 7 | 6 | 5 | 

Step 10: f(n) = 16, g(n) = 9, h(n) = 7 
| 2 | 8 | 1 | 
| * | 4 | 3 | 
| 7 | 6 | 5 | 

Step 11: f(n) = 18, g(n) = 10, h(n) = 8 
| * | 8 | 1 | 
| 2 | 4 | 3 | 
| 7 | 6 | 5 | 

Step 12: f(n) = 18, g(n) = 11, h(n) = 7 
| 8 | * | 1 | 

### A* Search using Nilsson Sequence Score as the Heuristic

In [13]:
iters_nilsson, search_cost_nilsson, result_nilsson = astar_search(start_puzzle, goal_puzzle, "nilsson")
print(f"Solution found for {iters_nilsson} iterations of A* search. The heuristic function is Nilsson Sequence Score.")
print(f"Search cost: {search_cost_nilsson} nodes generated")

Iteration 1:
f(n) = 54, g(n) = 0, h(n) = 54
Nilsson heuristic-specific values: P(n) = 12, S(n) = 14
Cumulative search cost = 4
Current node:
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Iteration 2:
f(n) = 51, g(n) = 1, h(n) = 50
Nilsson heuristic-specific values: P(n) = 11, S(n) = 13
Cumulative search cost = 7
Current node:
| 2 | 1 | 6 | 
| * | 4 | 8 | 
| 7 | 5 | 3 | 

Iteration 3:
f(n) = 51, g(n) = 1, h(n) = 50
Nilsson heuristic-specific values: P(n) = 11, S(n) = 13
Cumulative search cost = 10
Current node:
| 2 | 1 | 6 | 
| 4 | 8 | * | 
| 7 | 5 | 3 | 

Iteration 4:
f(n) = 51, g(n) = 2, h(n) = 49
Nilsson heuristic-specific values: P(n) = 10, S(n) = 13
Cumulative search cost = 12
Current node:
| 2 | 1 | * | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Iteration 5:
f(n) = 51, g(n) = 2, h(n) = 49
Nilsson heuristic-specific values: P(n) = 10, S(n) = 13
Cumulative search cost = 14
Current node:
| 2 | 1 | 6 | 
| 4 | 8 | 3 | 
| 7 | 5 | * | 

Iteration 6:
f(n) = 51, g(n) = 3, h(n) = 48
Nilsson heuristic-

In [14]:
solution_astar_nilsson = get_solution(result_nilsson)
i = 1
for node in solution_astar_nilsson:
    print(f"Step {i}: f(n) = {node.f}, g(n) = {node.g}, h(n) = {node.h} \n{node.puzzle}")
    i += 1

Step 1: f(n) = 54, g(n) = 0, h(n) = 54 
| 2 | 1 | 6 | 
| 4 | * | 8 | 
| 7 | 5 | 3 | 

Step 2: f(n) = 51, g(n) = 1, h(n) = 50 
| 2 | 1 | 6 | 
| 4 | 8 | * | 
| 7 | 5 | 3 | 

Step 3: f(n) = 51, g(n) = 2, h(n) = 49 
| 2 | 1 | * | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Step 4: f(n) = 53, g(n) = 3, h(n) = 50 
| 2 | * | 1 | 
| 4 | 8 | 6 | 
| 7 | 5 | 3 | 

Step 5: f(n) = 52, g(n) = 4, h(n) = 48 
| 2 | 8 | 1 | 
| 4 | * | 6 | 
| 7 | 5 | 3 | 

Step 6: f(n) = 49, g(n) = 5, h(n) = 44 
| 2 | 8 | 1 | 
| 4 | 6 | * | 
| 7 | 5 | 3 | 

Step 7: f(n) = 49, g(n) = 6, h(n) = 43 
| 2 | 8 | 1 | 
| 4 | 6 | 3 | 
| 7 | 5 | * | 

Step 8: f(n) = 49, g(n) = 7, h(n) = 42 
| 2 | 8 | 1 | 
| 4 | 6 | 3 | 
| 7 | * | 5 | 

Step 9: f(n) = 40, g(n) = 8, h(n) = 32 
| 2 | 8 | 1 | 
| 4 | * | 3 | 
| 7 | 6 | 5 | 

Step 10: f(n) = 37, g(n) = 9, h(n) = 28 
| 2 | 8 | 1 | 
| * | 4 | 3 | 
| 7 | 6 | 5 | 

Step 11: f(n) = 45, g(n) = 10, h(n) = 35 
| * | 8 | 1 | 
| 2 | 4 | 3 | 
| 7 | 6 | 5 | 

Step 12: f(n) = 45, g(n) = 11, h(n) = 34 
| 8 | * |

## Sanity Checking
Ensure that the solution by A* for each heuristic function is EXACTLY the same. Nothing should be printed after running the code block below.

In [15]:
solution_astar_misplaced = get_solution(result_misplaced)
solution_astar_manhattan = get_solution(result_manhattan)
solution_astar_nilsson = get_solution(result_nilsson)

for i in range(len(solution_astar_misplaced)):
    is_equal = solution_astar_misplaced[i] == solution_astar_manhattan[i] == solution_astar_nilsson[i]
    if not is_equal:
        print(f"There is an inconsistent solution among the three runs of A*")
        break